In [1]:
from transformers import Qwen2_5_VLForConditionalGeneration, AutoProcessor
from peft import PeftModel
import torch
import os
from PIL import Image
from tqdm import tqdm

C:\Users\aaron\PycharmProjects\QwenTesting\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# Carga del modelo oficial y el procesador

In [2]:
# Carga el modelo desde Hugging Face
#model = Qwen2_5_VLForConditionalGeneration.from_pretrained(
#    "Qwen/Qwen2.5-VL-7B-Instruct",
#    torch_dtype=torch.float16,
#    device_map="auto",
#)

# Usa el modelo cargado localmente
base_model = Qwen2_5_VLForConditionalGeneration.from_pretrained(
    "Qwen/Qwen2.5-VL-7B-Instruct",
    torch_dtype=torch.float16,
    device_map="auto",
)
model = PeftModel.from_pretrained(base_model, "../outputs_qwen2vl_finetuned/checkpoint-100")


# Procesador que combina tokenizer + preprocesamiento de imágenes
processor = AutoProcessor.from_pretrained("Qwen/Qwen2.5-VL-7B-Instruct")


Loading checkpoint shards: 100%|██████████| 5/5 [00:05<00:00,  1.10s/it]
Using a slow image processor as `use_fast` is unset and a slow processor was saved with this model. `use_fast=True` will be the default behavior in v4.52, even if the model was saved with a slow processor. This will result in minor differences in outputs. You'll still be able to use a slow processor with `use_fast=False`.


# Carga de imágenes locales y creación de mensajes

In [3]:
# Carpeta de imágenes local (ajusta la ruta según tu disco)
image_folder = "C:/Users/aaron/Desktop/Examenes/Inferencia"
all_files = sorted([f for f in os.listdir(image_folder) if f.endswith((".jpg", ".jpeg", ".png"))])

# Prompt personalizado
instruction = "Eres un profesor de educación primaria. Analiza y comenta si la actividad de la imagen está correcta o incorrecta; y, en caso de estar mal, explica cómo debería de haberse hecho."
max_new_tokens = 500


# Proceso de inferencia imagen por imagen

In [4]:
from qwen_vl_utils import process_vision_info

samples = []

for filename in tqdm(all_files):
    try:
        # Ruta absoluta del archivo
        image_path = os.path.join(image_folder, filename)

        # Crea el mensaje multimodal
        messages = [
            {"role": "user", "content": [
                {"type": "image", "image": f"file://{image_path}"},
                {"type": "text", "text": instruction}
            ]}
        ]

        # Aplica plantilla y procesa inputs
        text = processor.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)
        image_inputs, video_inputs = process_vision_info(messages)
        inputs = processor(
            text=[text],
            images=image_inputs,
            videos=video_inputs,
            padding=True,
            return_tensors="pt"
        ).to("cuda")

        # Genera respuesta
        with torch.no_grad():
            generated_ids = model.generate(
                **inputs,
                max_new_tokens=max_new_tokens,
                use_cache=True,
                temperature=1.5,
                do_sample=True,
                top_p=0.9
            )

        # Recorta tokens de entrada para obtener solo la respuesta
        trimmed_output = [
            output[len(input_ids):]
            for input_ids, output in zip(inputs.input_ids, generated_ids)
        ]

        # Decodifica respuesta
        decoded = processor.batch_decode(
            trimmed_output,
            skip_special_tokens=True,
            clean_up_tokenization_spaces=False
        )[0]

        print(f"\nImagen: {filename}")
        print(f"\nRespuesta del modelo: {decoded}")
        print("-" * 60)

        samples.append({
            "image": filename,
            "finetuned_output": decoded,
            #"base_output": decoded,
            #"expected_output": "",
        })

    except Exception as e:
        print(f"⚠️ Error con la imagen {filename}: {e}")


  1%|          | 1/102 [03:13<5:26:33, 194.00s/it]


Imagen: img1.jpg

Respuesta del modelo: La resolución propuesta es correcta: el alumno comprendió bien la cuestión (necesitaba calcular cuántas manzanas se pueden recoger multiplicando el número total de manzanos por el cantidad estimada que lleva cada uno), seleccionó correctamente la opción que incluye "una multiplicación", realizó el acoplamiento numérico 21 × 9 y obtuvo como resultado 189, lo cual es efectivamente la cantidad máxima de manzanas que podría recoger dadas esas circunstancias. Todo el cálculo está bien estructurado, refleja comprensión del contexto y operaciones apropiadas, junto con una breve interpretación final sobre la respuesta obtenida. Desde el punto de vista pedagógico, solo se recomendaría mejorar ligeramente la clareza ortográfica en algunas partes del ejercicio y mantener las anotaciones limpias y dentro del espacio proporcionado. La actividad ha sido ejecutada adecuadamente y no requiere cambios estructurales en su procedencia. Muy bien hecho al estudiante

  2%|▏         | 2/102 [03:53<2:51:29, 102.89s/it]


Imagen: img10.jpg

Respuesta del modelo: La actividad de multiplicación que muestra la imagen ha sido resuelta con un fallo técnico visible que desacredita su totalidad, siendo importante señalar el incidente para posteriormente corregir el ejercicio, pero primero debe repararse el problema de escritura. La ortodoxia numérica aparece superpuesta o dentro de un signo desconocido sobre “2316 × 5”, impidiendo evaluar ese dato en particular como correcta/incorrecta, porque es irrelevo una vez que el contenido visual oculta. Aunque se aprecia una anotación como si fueran tachones (cálculos incompletos) junto a las cifras superiores, no parece afectar especialmente la visibilidad del ejercicio, pero sí resta nitidez en la presentación general. El número escrito entre sí, a pesar del pequeño trastazón final del “10”, sería un “9616” si todas las cifras hubieran sido ajustadamente incluidas dentro del espacio proporcionado; por lo que esa sería la cifra teóricamente esperada correcta del prod

  3%|▎         | 3/102 [04:18<1:51:43, 67.71s/it] 

...


Imagen: img7.jpg

Respuesta del modelo: La multiplicación propuesta (8012 × 4) ha sido efectuado con error, siendo el resultado presentado por el alumno “32,048”, lo cual es erróneo. El resultado correcto sería “32.048”. Aun así, hay múltiples errores en la presentación del cálculo, empezando por la letra de escritura que sobresale del borde superior derecho de la figura, lo que suele reflejar falta de cuidado en la ortografía y presenta aspectos poco visuales. Lo más preocupante, sin embargo, está en la operación numérica realizada, puesto que falla comprensión básica al entender que las cifras son unidades irreductibles, y aunque esto podría apuntar una deficiente organización general sobre este tipo de calculabilidad es posible observar intenciones correctas del alumno a pesar de sus fallos ortográficos. Recomendaciones pedagógicas específicas girarían tanto en acentuar la precisión inicial como el formato de respuesta final, como intensificar la reiterativa revisión de conceptos n

 99%|█████████▉| 101/102 [5:01:59<00:30, 30.29s/it]


Imagen: img8.jpg

Respuesta del modelo: La operación mostrada es una multiplicación. El alumnadebió calcular el producto 1634 × 9. El número escrito arriba está correcto. Como se puede ver, se comprobó su resultado previamente con un lápiz (se ha marcado X), pero no acertó. El producto correcto del cálculo requerido es 14706, y no 16206, que es el número escrito abajo. Por tanto, esta vez tampoco acertó. Ha sido útil poner como ejemplo un cálculo error entre sí y también la verificación externa. La redacción matemática se mantiene ordenada y grande, lo cual mejora la comprensión. Los números se entienden con seguridad y tienen tamaño acorde al tamaño de las unidades, siendo claro lo escrito por tanto. Sin embargo, convendría revisar si aparece alguna confusión con letras grandes o desproporcionadas que dificultan su reconocimiento por completo. Lo recomendable sería insistir especialmente en revisar los aciertos obtenidos junto con errores comunes, fijar recompensas parciales como ano

100%|██████████| 102/102 [5:02:36<00:00, 178.01s/it]


Imagen: img9.jpg

Respuesta del modelo: La imagen muestra una multiplicación por una unidad que ha sido realizada correctamente: 3107 × 8 = 24856. El alumno, sin embargo, cometió un error de escritura durante el desarrollo: dejó un trazo en la parte superior del número de abajo (la cifra 3), lo cual podría alterar la claridad al comprobar esa columna específica de cálculo, aunque no impide ni afecta a la validación final del resultado matemático. Por tanto, los números están bien ordenados, el número inicial es factible, el acoplamiento de las unidades con la decena inferior es perfectamente comprensible si se hace una revisión rápida con criterio docente. La respuesta anota como correcta y debe entenderse como tal de forma didáctica y sin castigar errores decorativos indeseables, centrando la retroalimentación docente puramente sobre el procedimiento o resultados cuantitativos de error en su plenitud (compréhnsibles para los estudiantes). Sin duda se recomienda prestar especial atenc

# Guardar resultados en JSON

In [5]:
import json

output_path = "C:/Users/aaron/Desktop/Final/finetune_qwen_inference.json"

with open(output_path, "w", encoding="utf-8") as f:
    json.dump(samples, f, ensure_ascii=False, indent=2)

print(f"\n✅ Resultados guardados en: {output_path}")



✅ Resultados guardados en: C:/Users/aaron/Desktop/Final/finetune_qwen_inference.json


In [6]:
model.save_pretrained("models/qwen2vl_local")
processor.save_pretrained("models/qwen2vl_local")

[]